使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import s04_07_h_max_capacity_of_hot_water_floor_heating as appendix_H

import numpy as np

## 付録D　電気ヒートポンプ温水暖房機（フロン系） / Appendix D Electric heat pump water heater (fluorocarbon type)

### D.1 記号及び単位 / Symbol and unit

省略

ommision

### D.2 エネルギー消費量 / Energy consumption

#### D.2.1 消費電力量 / Power consumption

$$\large
E_{E,hs} = \frac{Q_{out,H,hs}}{e_{hs}} \times 10^{3} \div 3600
$$

$E_{E,hs}$：1 時間当たりの温水暖房用熱源機の消費電力量(kWh/h) / Power consumption of the hot water heating heat source unit per hour  
$Q_{out,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h) / Heating output of the heat source machine for hot water heating per hour  
$e_{hs}$：1 時間平均の温水暖房用熱源機の効率 / 1 hour average hot water heating heat source machine efficiency

In [ ]:
# 1時間当たりの消費電力量 (1)
def get_E_E_hs(Q_out_H_hs, Q_max_H_hs, Q_dmd_H_hs_d_t, Theta_SW_hs, Theta_ex, q_rtd_hs):
    # 定格効率
    e_rtd = get_e_rtd()

    # 定格消費電力
    P_rtd_hs = get_P_rtd_hs(q_rtd_hs, e_rtd)

    # 1 時間平均の温水暖房用熱源機の効率
    e_hs = get_e_hs(Q_max_H_hs, Theta_SW_hs, Theta_ex, Q_out_H_hs, P_rtd_hs)

    # 消費電力量
    E_E_hs = Q_out_H_hs / e_hs * 10 ** 3 / 3600
    E_E_hs[Q_dmd_H_hs_d_t == 0] = 0

    return E_E_hs

$$\large
e_{hs} = \frac{Q_{max,H,hs} \times 10^{6} \div 3600}{P_{max,hs}} \times er_{hs}
$$

$Q_{max,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h) / Heating output of the heat source machine for hot water heating per hour  
$P_{max,hs}$：温水暖房用熱源機の最大消費電力(W) / Maximum power consumption of heat source equipment for hot water heating  
$er_{hs}$：1 時間平均の温水暖房用熱源機の効率比 / 1 hour average efficiency ratio of hot water heating heat source machine

In [ ]:
# 1 時間平均の温水暖房用熱源機の効率 (2)
def get_e_hs(Q_max_H_hs, Theta_SW_hs, Theta_ex, Q_out_H_hs, P_rtd_hs):

    # 1時間平均の効率比
    e_r_hs = get_e_r_hs(Theta_SW_hs, Theta_ex, Q_out_H_hs, Q_max_H_hs)

    # 最大消費電力
    P_max_hs = get_P_max_hs(P_rtd_hs)

    return (Q_max_H_hs * 10 ** 6 / 3600) / P_max_hs * e_r_hs

$$\large
P_{max,hs} = \frac{P_{rtd,hs}}{0.6}
$$

$P_{rtd,hs}$：温水暖房用熱源機の定格消費電力(W) / Rated power consumption of heat source equipment for hot water heating  

In [ ]:
# 最大消費電力 (3)
def get_P_max_hs(P_rtd_hs):
    return P_rtd_hs / 0.6

$$\large
P_{rtd,hs} = \frac{q_{rtd,hs}}{e_{rtd}}
$$

$q_{rtd,hs}$：温水暖房用熱源機の定格能力(W) / Rated capacity of hot water heating heat source machine  
$e_{rtd}$：温水暖房用熱源機の定格効率 / Rated efficiency of the hot water heating heat source machine

In [ ]:
# 定格消費電力 (4)
def get_P_rtd_hs(q_rtd_hs, e_rtd):
    return q_rtd_hs / e_rtd

温水暖房用熱源機の定格効率$e_{rtd}$は4.05とする。

The rated efficiency $ e_{rtd} $ of the hot water heating heat source unit is 4.05.

In [ ]:
# 定格効率
def get_e_rtd():
    return 4.05

$$\large
q_{rtd,hs} = q_{max,hs} \times 0.8
$$

$q_{max,hs}$：温水暖房用熱源機の最大能力(W) / Maximum capacity of heat source equipment for hot water heating  

In [ ]:
# 定格能力 (5)
def get_q_rtd_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    # 最大能力
    q_max_hs = appendix_H.get_q_max_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

    return q_max_hs * 0.8

$$\large
er_{hs} = \big(1.120656 - 0.03703 \times (\theta_{SW,hs} - \theta_{ex})\big) \times \bigg(1 - \frac{Q_{out,H,hs}}{Q_{max,H,hs}}\bigg)^{2}\\
\large + \big(-0.36786 + 0.012152 \times (\theta_{SW,hs} - \theta_{ex})\big) \times \bigg(1 - \frac{Q_{out,H,hs}}{Q_{max,H,hs}}\bigg)\\
\large + 1
$$

$\theta_{SW,hs}$：1 時間平均の温水暖房用熱源機の往き温水温度(℃) / 1 hour average hot water temperature of the hot water source heater for hot water heating  
$\theta_{ex}$：1 時間平均の外気温度(℃) / 1 hour average outside temperature

In [ ]:
# 1時間平均の効率比 (6)
def get_e_r_hs(Theta_SW_hs, Theta_ex, Q_out_H_hs, Q_max_H_hs):
    return (1.120656 - 0.03703 * (Theta_SW_hs - Theta_ex)) * (1 - Q_out_H_hs / Q_max_H_hs) ** 2 \
           + (-0.36786 + 0.012152 * (Theta_SW_hs - Theta_ex)) * (1 - Q_out_H_hs / Q_max_H_hs) \
           + 1

#### D.2.2 ガス消費量 / Gas consumption

1 時間当たりの温水暖房用熱源機のガス消費量$E_{G,hs}$は0とする。

The gas consumption $ E_{G, hs} $ of the hot water heating heat source unit per hour is assumed to be zero.

In [ ]:
# ガス消費量
def get_E_G_hs():
    return np.zeros(24 * 365)

#### D.2.3 石油消費量 / Oil consumption

1 時間当たりの温水暖房用熱源機の石油消費量$E_{K,hs}$は0とする。

The oil consumption $ E_{K, hs} $ of the hot water heating heat source unit per hour is 0.

In [ ]:
# 灯油消費量
def get_E_K_hs():
    return np.zeros(24 * 365)

#### D.2.4 その他の一次エネルギー消費量 / Other primary energy consumption

1 時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量$E_{M,hs}$は0とする。

The other primary energy consumption $ E_{M, hs} $ of the hot water heating heat source unit per hour is assumed to be zero.

In [ ]:
# その他の一次エネルギー消費量
def get_E_M_hs():
    return np.zeros(24 * 365)

### D.3 最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,hs} = (11.62 + 0.2781 \times \theta_{ex} - 0.00113 \times \theta_{ex}^{2}\\
\large -0.1271 \times \theta_{SW,hs} - 0.00363 \times \theta_{ex} \times \theta_{SW,hs})
\times \frac{q_{rtd,hs}}{6} \times \frac{C_{def}}{0.85} \times 3600 \times 10^{-6}
$$

$C_{def}$：1 時間あたりのデフロスト補正係数 / Hourly Defrost Correction Factor

In [ ]:
# 最大暖房出力 (7)
def get_Q_max_H_hs(q_rtd_hs, Theta_SW_hs, Theta_ex, h_ex):
    # デフロスト補正係数
    C_def = get_C_def(Theta_ex, h_ex)

    return (
                       11.62 + 0.2781 * Theta_ex - 0.00113 * Theta_ex ** 2 - 0.1271 * Theta_SW_hs - 0.00363 * Theta_ex * Theta_SW_hs) \
           * (q_rtd_hs / 6) * (C_def / 0.85) * 3600 * 10 ** (-6)

1 時間あたりのデフロスト補正係数$C_{def}$は、1 時間平均の外気温度$\theta_{ex}$が5℃ 未満かつ 1 時間平均の外気相対湿度$h_{ex}$が80%以上の場合は0.85とし、それ以外の場合は1.0とする。 

The hourly defrost correction coefficient $ C_{def} $ is an hourly average outdoor temperature $ \theta_{ex} $ less than 5 ° C and an hourly average outdoor relative humidity $ h_{ex} $ 80% or more In the case of 0.85, and in other cases 1.0.

In [ ]:
# デフロスト補正係数
def get_C_def(Theta_ex, h_ex):
    C_def = np.zeros(24 * 365)
    C_def[:] = 1
    C_def[(Theta_ex < 5) * (h_ex >= 80)] = 0.85
    return C_def